In [368]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Serif'
plt.rcParams['font.size'] = 15

In [369]:
df_heart_1 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_1.csv")
df_heart_2 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_2.csv")
df_heart_3 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_3.csv")
df_heart_6 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_6.csv")

data_1 = pd.concat([df_heart_1, df_heart_2, df_heart_3, df_heart_6],ignore_index=True)
# Drop the 'thal' column
data_1.drop(['thal', 'ca', 'Unnamed: 0'], axis=1, inplace=True)
# Rename multiple columns
data_1.rename(columns={'cp': 'chest pain', 'trestbps': 'resting bps', 'chol': 'cholesterol', 
                       'fbs': 'fasting blood sugar', 'restecg': 'resting ecg', 'thalach': 'max heart rate',
                       'exang': 'exercise angina', 'target': 'disease'}, inplace=True)

df_heart_4 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_4.csv")
df_heart_5 = pd.read_csv("./datasets/heart_disease/heart_disease_dataset_5.csv")
df_heart_5.rename(columns={'resting bp s': 'resting bps'}, inplace=True)

data_2 = pd.concat([df_heart_4, df_heart_5],ignore_index=True)
data_2.drop(['Unnamed: 0'], axis=1, inplace=True)
data_2.rename(columns={'ST slope': 'slope', 'target': 'disease', 'chest pain type': 'chest pain'}, inplace=True)

df_heart_di = pd.concat([data_1, data_2],ignore_index=True)

mapping = {0: 'non-heart disease', 1: 'heart disease'}
df_heart_di['disease'] = df_heart_di['disease'].map(mapping)

df_heart_di.to_csv('./datasets/heart_disease/final_heart_dataset.csv', index=False)

df_heart_di


,age,sex,chest pain,resting bps,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,slope,disease
0,52,1,0,125,212.0,0,1,168,0,1.0,2,non-heart disease
1,53,1,0,140,203.0,1,0,155,1,3.1,0,non-heart disease
2,70,1,0,145,174.0,0,1,125,1,2.6,0,non-heart disease
3,61,1,0,148,203.0,0,1,161,0,0.0,2,non-heart disease
4,62,0,0,138,294.0,1,1,106,0,1.9,1,non-heart disease
...,...,...,...,...,...,...,...,...,...,...,...,...
4166,45,1,1,110,264.0,0,0,132,0,1.2,2,heart disease
4167,68,1,4,144,193.0,1,0,141,0,3.4,2,heart disease
4168,57,1,4,130,131.0,0,0,115,1,1.2,2,heart disease
4169,57,0,2,130,236.0,0,2,174,0,0.0,2,heart disease


In [370]:
df_diabetes_1 = pd.read_csv("./datasets/diabetes/diabetes_dataset_1.csv")
df_diabetes_2 = pd.read_csv("./datasets/diabetes/diabetes_dataset_2.csv")
df_diabetes_3 = pd.read_csv("./datasets/diabetes/diabetes_dataset_3.csv")
df_diabetes_4 = pd.read_csv("./datasets/diabetes/diabetes_dataset_4.csv")
df_diabetes_5 = pd.read_csv("./datasets/diabetes/diabetes_dataset_5.csv")
df_diabetes_5 = df_diabetes_5.rename(columns={'Diagnosis': 'Outcome'}, inplace=True)
df_diabetes = pd.concat([df_diabetes_1, df_diabetes_2, df_diabetes_3, df_diabetes_4, df_diabetes_5],ignore_index=True)
df_diabetes.rename(columns={'Outcome': 'disease', 'Age': 'age'}, inplace=True)

mapping = {0: 'non-diabetics', 1: 'diabetes'}
df_diabetes['disease'] = df_diabetes['disease'].map(mapping)

df_diabetes.to_csv('./datasets/diabetes/final_diabetes_dataset.csv', index=False)

df_diabetes

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,age,disease
0,6,148,72,35,0,33.6,0.627,50,diabetes
1,1,85,66,29,0,26.6,0.351,31,non-diabetics
2,8,183,64,0,0,23.3,0.672,32,diabetes
3,1,89,66,23,94,28.1,0.167,21,non-diabetics
4,0,137,40,35,168,43.1,2.288,33,diabetes
...,...,...,...,...,...,...,...,...,...
5531,2,75,64,24,55,29.7,0.370,33,non-diabetics
5532,8,179,72,42,130,32.7,0.719,36,diabetes
5533,6,85,78,0,0,31.2,0.382,42,non-diabetics
5534,0,129,110,46,130,67.1,0.319,26,diabetes


In [371]:
df_anemia_3 = pd.read_csv("./datasets/anemia/anemia_dataset_3.csv")
selected_columns = ['GENDER', 'HGB','MCV', 'MCH', 'MCHC', 'HGB_Anemia_Class'] 
df_anemia_3 = df_anemia_3[selected_columns]
df_anemia_3.rename(columns={'GENDER': 'sex', 'HGB': 'hemoglobin', 'HGB_Anemia_Class': 'disease'}, inplace=True)
df_anemia_3[df_anemia_3['disease'] == 1]

df_anemia_1 = pd.read_csv("./datasets/anemia/anemia_dataset_1.csv")
df_anemia_2 = pd.read_csv("./datasets/anemia/anemia_dataset_2.csv")
data1_2 = pd.concat([df_anemia_1, df_anemia_2],ignore_index=True)
data1_2.rename(columns={'Gender': 'sex', 'Hemoglobin': 'hemoglobin', 'Result': 'disease'}, inplace=True)

df_anemia_4 = pd.read_csv("./datasets/anemia/anemia_dataset_4.csv")
df_anemia_4.rename(columns={'Outcome': 'disease'}, inplace=True)

df_anemia_di = pd.concat([df_anemia_3, data1_2, df_anemia_4],ignore_index=True)

mapping = {0: 'non-hgb-anemia', 1: 'hgb_anemia'}
df_anemia_di['disease'] = df_anemia_di['disease'].map(mapping)

df_anemia_di.to_csv('./datasets/anemia/final_anemia_dataset.csv', index=False)

df_anemia_di

,sex,hemoglobin,MCV,MCH,MCHC,disease,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,AnemiaPedigreeFunction,age
0,1.0,12.7,87.2,29.5,33.8,non-hgb-anemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,12.8,85.0,29.5,34.7,non-hgb-anemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,9.4,86.5,29.6,34.2,non-hgb-anemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,10.5,93.7,31.3,33.4,non-hgb-anemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,5.1,109.2,38.9,35.7,non-hgb-anemia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26028,NaN,NaN,NaN,NaN,NaN,non-hgb-anemia,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0
26029,NaN,NaN,NaN,NaN,NaN,non-hgb-anemia,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0
26030,NaN,NaN,NaN,NaN,NaN,non-hgb-anemia,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0
26031,NaN,NaN,NaN,NaN,NaN,hgb_anemia,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0


In [382]:
data = pd.concat([df_heart_di, df_diabetes, df_anemia_di],ignore_index=True)
data

,age,sex,chest pain,resting bps,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,...,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,hemoglobin,MCV,MCH,MCHC,AnemiaPedigreeFunction
0,52.0,1.0,0.0,125.0,212.0,0.0,1.0,168.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.0,1.0,0.0,140.0,203.0,1.0,0.0,155.0,1.0,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,70.0,1.0,0.0,145.0,174.0,0.0,1.0,125.0,1.0,2.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61.0,1.0,0.0,148.0,203.0,0.0,1.0,161.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,62.0,0.0,0.0,138.0,294.0,1.0,1.0,106.0,0.0,1.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35735,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,76.0,48.0,180.0,32.9,NaN,NaN,NaN,NaN,NaN,0.171
35736,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,70.0,27.0,0.0,36.8,NaN,NaN,NaN,NaN,NaN,0.340
35737,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,72.0,23.0,112.0,26.2,NaN,NaN,NaN,NaN,NaN,0.245
35738,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60.0,0.0,0.0,30.1,NaN,NaN,NaN,NaN,NaN,0.349


In [ ]:
# Let's find how many of each class there are in the bar chart.
counts = data['disease'].value_counts()
palette = sns.color_palette()
fig, axes = plt.subplots(figsize=(10,6))
counts.plot(kind="bar", ax=axes, color=palette)
plt.xlabel('diseases',labelpad=10)
for container in axes.containers:
    axes.bar_label(container)
plt.show()

In [ ]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error, make_scorer, precision_score, f1_score, confusion_matrix, classification_report,accuracy_score, log_loss, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
disease=data['disease']
features=data.drop('disease',axis=1)
X_train, X_test, Y_train, Y_test=train_test_split(features,disease,test_size=0.2, shuffle=True, random_state=42, stratify=disease)
# Create an imputer object with a chosen strategy (e.g., mean)
imputer = SimpleImputer(strategy='mean')
# Fit the imputer on your training data and transform it
X_train_imputed = imputer.fit_transform(X_train)
# Transform the test data using the imputer fitted on the training data
X_test_imputed = imputer.transform(X_test)

In [ ]:
KN = KNeighborsClassifier()
LR = LogisticRegression()
DT = DecisionTreeClassifier()
svc = SVC()
RF = RandomForestClassifier()
GB = GradientBoostingClassifier()
algorithms = [KN,LR,DT,svc,RF,GB]

In [ ]:
def compare_algorithms(X_train, y_train, algorithms, cv=10, scoring='accuracy'):
    """
    Compare the accuracies of multiple machine learning algorithms using cross-validation.

    Parameters:
    - X: Features (input variables) as a numpy array or pandas DataFrame.
    - y: Target variable as a numpy array or pandas Series.
    - algorithms: List of machine learning algorithms to compare.
    - cv: Number of cross-validation folds (default=10).
    - scoring: Evaluation metric to use (default='accuracy').

    Returns:
    - accuracies: Dictionary containing the accuracies of each algorithm.
    """

    accuracies = {}

    for algorithm in algorithms:
        algorithm.fit(X_train, y_train)
        scores = cross_val_score(algorithm, X_test_imputed, Y_test, cv=cv, scoring=scoring)
        accuracies[type(algorithm).__name__] = np.mean(scores)

    return accuracies

In [ ]:
def plot_accuracies(accuracies):
    """
    Plot a bar graph of accuracies for each algorithm.

    Parameters:
    - accuracies: Dictionary containing the accuracies of each algorithm.
    """

    algorithms = list(accuracies.keys())
    scores = list(accuracies.values())

    plt.figure(figsize=(10, 6))
    plt.barh(algorithms, scores, color='skyblue')
    plt.xlabel('Accuracy')
    plt.ylabel('Algorithm')
    plt.title('Accuracy of Machine Learning Algorithms')
    plt.xlim(0, 1)  # Set x-axis limits to range from 0 to 1
    plt.show()

In [ ]:
# Compare algorithms and plot accuracies
accuracies = compare_algorithms(X_train_imputed, Y_train, algorithms)
plot_accuracies(accuracies)

In [ ]:
print("Accuracies:", accuracies)

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_imputed, Y_train)

# Predict on the testing set
y_pred = rf_classifier.predict(X_test_imputed)

print('accuracy', accuracy_score(Y_test, y_pred))

In [ ]:
# Evaluate the model
print("Classification Report:\n", classification_report(Y_test, y_pred))

In [ ]:
print("\nConfusion Matrix:")
print(confusion_matrix(Y_test, y_pred))

In [ ]:
from tensorflow.keras.callbacks import TensorBoard

label_encoder = LabelEncoder()
Y_train_encoded = label_encoder .fit_transform(Y_train)

# Create a TensorBoard callback
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)), Dropout(0.2),
    Dense(32, activation='relu'), Dropout(0.2),
    Dense(1, activation='sigmoid')
])

optimizer = Adam(learning_rate=0.01)  # Adjust learning rate here
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_scaled, Y_train_encoded, epochs=50, batch_size=32, validation_split=0.1, callbacks=[tensorboard], verbose=1)